In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

# Start SparkSession
spark = SparkSession.builder.appName("CSV Load Example").master("local[*]").getOrCreate()

# Load CSV file into DataFrame
df = spark.read.csv("C:/Users/andre/OneDrive/Dokument/vscode/Examens/aapl_10min_2yr.csv", header=True, inferSchema=True)
# Show the first few rows
df.show()

# Print the schema (column types)
df.printSchema()

+-------------------+-------+-------+-------+-------+-------+
|               date|   open|   high|    low|  close| volume|
+-------------------+-------+-------+-------+-------+-------+
|2023-04-24 15:30:00|164.975| 165.24|164.415| 165.06|60755.0|
|2023-04-24 15:40:00| 165.04| 165.46| 164.94| 165.13|14699.0|
|2023-04-24 15:50:00| 165.13|165.415|165.075| 165.17| 6410.0|
|2023-04-24 16:00:00|165.175| 165.18| 164.82| 164.97|11225.0|
|2023-04-24 16:10:00|164.955| 165.28| 164.82| 165.26|15571.0|
|2023-04-24 16:20:00| 165.26|165.425|165.165|165.295|10654.0|
|2023-04-24 16:30:00|165.285| 165.36| 164.47|  164.5| 8380.0|
|2023-04-24 16:40:00|164.495| 164.52|164.165| 164.25|12247.0|
|2023-04-24 16:50:00| 164.25|  164.4| 164.02|164.035|13455.0|
|2023-04-24 17:00:00| 164.04| 164.23|  163.9| 164.15| 8376.0|
|2023-04-24 17:10:00|164.145| 164.38| 164.07|164.355| 8748.0|
|2023-04-24 17:20:00| 164.36| 164.41| 164.15| 164.17| 6981.0|
|2023-04-24 17:30:00| 164.17| 164.33|  164.1|164.135|12273.0|
|2023-04

In [3]:
spark.stop()